In [1]:
#set up 3 folders 'panel','vp','output'  
#the former 2 containing the input csv with the same filename as in file
#######
#bring all geometry close to origin to improve quality of data output
#the input csv should be, for panels: 'x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,y4,z4', without headers. 
#x,y,z coordinates of vertexes 1,2,3,(4). vertex 4 should be coincident with vertex 3 in case panels are triangular.
#######
#the input csv should be, for viewpoints: 'xvp,yvp,zvp', without headers.

import pandas as pd
import numpy as np

#set directory and file 
file='prova'
panel_dir="C:/Users/florio/Desktop/prova/sa/panel/"
vp_dir='C:/Users/florio/Desktop/prova/sa/vp/'
output_dir='C:/Users/florio/Desktop/prova/sa/output/'
print('input panel dir: '+panel_dir+file+'.csv')
print('viewpoints dir: '+vp_dir+file+'.csv')
print('output mean solid angle per panel across viewpoints dir: '+output_dir+file+'.res')

#read csv and extract vertex coordinates column
tile=pd.read_csv(panel_dir+file+'.csv',header=None)
coord=tile #subset CSV table if needed
#print(coord)

input panel dir: C:/Users/florio/Desktop/prova/sa/panel/prova.csv
viewpoints dir: C:/Users/florio/Desktop/prova/sa/vp/prova.csv
output mean solid angle per panel across viewpoints dir: C:/Users/florio/Desktop/prova/sa/output/prova.res


In [2]:
#split coordinates into x y and z values
splitted_coord=coord
#label columns
cols=['x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4']
splitted_coord.columns=cols
print(splitted_coord)

           x1         y1         z1         x2         y2         z2  \
0   59.386353  38.201061  12.422496  52.887817  42.414249  12.422496   
1   59.386353  38.201061   4.677698  52.887817  42.414249   4.677698   
2   65.884888  33.987877  12.422496  59.386353  38.201061  12.422496   
3   65.884888  33.987877   4.677698  59.386353  38.201061   4.677698   
4   49.493889  44.614632  21.994125  52.887817  42.414249  20.167294   
5   52.887817  42.414249  20.167294  48.067638  45.539310  19.397287   
6   52.887817  42.414249  20.167294  49.493889  44.614632  21.994125   
7   47.135994  46.143322  16.601015  52.887817  42.414249  12.422496   
8   47.135994  46.143322  16.601015  47.538784  45.882183  18.012337   
9   48.067638  45.539310  19.397287  52.887817  42.414249  20.167294   
10  46.856857  46.324295  15.185764  52.887817  42.414249  12.422496   
11  52.887817  42.414249  20.167294  50.376240  44.042580  23.160534   
12  57.067741  39.704285  27.354851  59.386353  38.201061  20.16

In [3]:
#read viewpoints csv
viewpoints=pd.read_csv(vp_dir+file+'.csv',header=None)
viewpoints.columns=['xvp','yvp','zvp']
vpid=viewpoints.index.values
print(viewpoints)

         xvp        yvp        zvp
0  49.173159  26.963185  12.422495


In [18]:
#create empty matrix to host solid angles, panels in rows and viewpoints in columns
sa_matrix=np.empty((splitted_coord.shape[0],viewpoints.shape[0]))
#calculate solid angle between viewpoint and panels
import time
import datetime
start_time = time.time()
for nvp, vp in viewpoints.iterrows():
    a=np.array([splitted_coord['x1']-vp['xvp'],splitted_coord['y1']-vp['yvp'],splitted_coord['z1']-vp['zvp']]).T.tolist()
    b=np.array([splitted_coord['x2']-vp['xvp'],splitted_coord['y2']-vp['yvp'],splitted_coord['z2']-vp['zvp']]).T.tolist()
    c=np.array([splitted_coord['x3']-vp['xvp'],splitted_coord['y3']-vp['yvp'],splitted_coord['z3']-vp['zvp']]).T.tolist()
    d=np.array([splitted_coord['x4']-vp['xvp'],splitted_coord['y4']-vp['yvp'],splitted_coord['z4']-vp['zvp']]).T.tolist()   
    vectors=pd.DataFrame({'veca':a,'vecb':b,'vecc':c,'vecd':d})
    def ab(row):
        return np.dot(row['veca'],row['vecb'])  
    def ac(row):
        return np.dot(row['veca'],row['vecc'])
    def bc(row):
        return np.dot(row['vecb'],row['vecc'])
    def bxc(row):
        return np.cross(row['vecb'],row['vecc']).tolist()
    def ad(row):
        return np.dot(row['veca'],row['vecd'])
    def cd(row):
        return np.dot(row['vecc'],row['vecd'])
    def cxd(row):
        return np.cross(row['vecc'],row['vecd']).tolist()
    vectors['ab']=vectors.apply(ab,axis=1)
    vectors['ac']=vectors.apply(ac,axis=1)
    vectors['bc']=vectors.apply(bc,axis=1)
    vectors['bxc']=vectors.apply(bxc,axis=1)
    vectors['ad']=vectors.apply(ad,axis=1)
    vectors['cd']=vectors.apply(cd,axis=1)
    vectors['cxd']=vectors.apply(cxd,axis=1)
    def sa(row):
        return np.abs(2*np.arctan(np.dot(row['veca'],row['bxc'])/(np.linalg.norm(row['veca'])*np.linalg.norm(row['vecb'])*np.linalg.norm(row['vecc'])+row['ab']*np.linalg.norm(row['vecc'])+row['ac']*np.linalg.norm(row['vecb'])+row['bc']*np.linalg.norm(row['veca']))))+np.abs(2*np.arctan(np.dot(row['veca'],row['cxd'])/(np.linalg.norm(row['veca'])*np.linalg.norm(row['vecd'])*np.linalg.norm(row['vecc'])+row['ad']*np.linalg.norm(row['vecc'])+row['ac']*np.linalg.norm(row['vecd'])+row['cd']*np.linalg.norm(row['veca']))))
    vectors['sa']=vectors.apply(sa,axis=1)
    sa_matrix[:,nvp]=vectors['sa']
    print ('\r'+str(round(nvp/len(viewpoints)*100,2))+' % complete', end='')
elapsed=(time.time() - start_time)
print("--- %s seconds ---" % elapsed)
print(str(datetime.timedelta(seconds=elapsed)))

0.0 % complete--- 0.08099985122680664 seconds ---
0:00:00.081000


In [15]:
#check matrix
print(sa_matrix)
print(sa_matrix.shape)

[[ 0.22772799]
 [ 0.22772801]
 [ 0.18699166]
 [ 0.18699168]
 [ 0.00726389]
 [ 0.00768869]
 [ 0.00700071]
 [ 0.01399204]
 [ 0.01473054]
 [ 0.0082599 ]
 [ 0.01335487]
 [ 0.00690418]
 [ 0.01134361]
 [ 0.010615  ]
 [ 0.01000583]
 [ 0.00696901]
 [ 0.00718056]
 [ 0.06248574]
 [ 0.01220646]
 [ 0.00751666]
 [ 0.07091631]
 [ 0.00985   ]
 [ 0.01335488]
 [ 0.01473055]
 [ 0.01399202]
 [ 0.00825991]
 [ 0.0072639 ]
 [ 0.00768868]
 [ 0.00700071]
 [ 0.01281205]
 [ 0.00696901]
 [ 0.00690418]
 [ 0.07091631]
 [ 0.00718056]
 [ 0.00751666]
 [ 0.01220646]
 [ 0.01134361]
 [ 0.010615  ]
 [ 0.06248576]
 [ 0.01281204]
 [ 0.01047065]
 [ 0.0109022 ]
 [ 0.05220878]
 [ 0.00546053]
 [ 0.04699283]
 [ 0.00515206]
 [ 0.00496932]
 [ 0.00588629]
 [ 0.00498469]
 [ 0.00516998]
 [ 0.00545651]
 [ 0.00878801]
 [ 0.00798396]
 [ 0.00834347]
 [ 0.00932835]
 [ 0.00491475]
 [ 0.00798397]
 [ 0.00588629]
 [ 0.05220881]
 [ 0.00546054]
 [ 0.00515207]
 [ 0.00985001]
 [ 0.01047065]
 [ 0.01012227]
 [ 0.01090221]
 [ 0.01000583]
 [ 0.04699

In [16]:
#calculate mean of values per row (panel) across the different columns (viewpoints)
#calculate sum of row values, i.e. the total solid angle subtended by all panels
sa_mean = np.mean(sa_matrix,axis=1)
sa_sum=np.sum(sa_matrix,axis=0)
print('average solid angle: '+str(sa_mean))
print('average solid angle array shape: '+str(sa_mean.shape))
print('total solid angle of different panels: '+str(sa_sum))


average solid angle: [ 0.22772799  0.22772801  0.18699166  0.18699168  0.00726389  0.00768869
  0.00700071  0.01399204  0.01473054  0.0082599   0.01335487  0.00690418
  0.01134361  0.010615    0.01000583  0.00696901  0.00718056  0.06248574
  0.01220646  0.00751666  0.07091631  0.00985     0.01335488  0.01473055
  0.01399202  0.00825991  0.0072639   0.00768868  0.00700071  0.01281205
  0.00696901  0.00690418  0.07091631  0.00718056  0.00751666  0.01220646
  0.01134361  0.010615    0.06248576  0.01281204  0.01047065  0.0109022
  0.05220878  0.00546053  0.04699283  0.00515206  0.00496932  0.00588629
  0.00498469  0.00516998  0.00545651  0.00878801  0.00798396  0.00834347
  0.00932835  0.00491475  0.00798397  0.00588629  0.05220881  0.00546054
  0.00515207  0.00985001  0.01047065  0.01012227  0.01090221  0.01000583
  0.04699283  0.00932836  0.00545652  0.00496932  0.00491474  0.0049847
  0.00516997  0.008788    0.00834348  0.01012226]
average solid angle array shape: (76,)
total solid angl

In [50]:
#export as textfiles
sa_mean_pd=pd.DataFrame({'sa_mean_per_vp':sa_mean})
sa_mean_pd.to_csv(output_dir+file+".res")
print("export complete")

export complete
